In [1]:
import csv
import tensorflow as tf
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

### Train Batch Pre-Processing

In [2]:
vocabulary_size = 8000
unknown_token = "UNKNOWN_TOKEN"

with open('Interviewdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    questions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(questions))
with open('Interviewdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    answers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(answers))
with open('Interviewdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    labels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(labels)) 

408
408
408


In [3]:
import pickle
with open ('words.txt', 'rb') as fp:
    wordlist = pickle.load(fp)

In [4]:
with open('embeddings.txt','rb') as fp:
    embeddings = pickle.load(fp)

In [5]:
wordlist.append('UNKNOWN_TOKEN')

In [6]:
len(wordlist)

1720

In [7]:
embeddings = np.vstack([embeddings, np.zeros(200)])

In [8]:
len(embeddings)

1720

In [9]:
word_to_index = dict([(w,i) for i,w in enumerate(wordlist)])

In [10]:
len(word_to_index)

1720

In [11]:
for i, sent in enumerate(questions):
    questions[i] = [w if w in word_to_index else unknown_token for w in sent]

In [12]:
for i, sent in enumerate(answers):
    answers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [13]:
import numpy as np
X_question = np.asarray([[word_to_index[w] for w in sent] for sent in questions])
y_answer = np.asarray([[word_to_index[w] for w in sent] for sent in answers])

In [14]:
X_question[:10]

array([ [5, 8, 184, 0, 72, 2, 41, 447, 136, 122, 878, 136, 122, 258, 184, 1111, 389, 362, 1228, 7],
       [5, 8, 184, 0, 72, 2, 19, 47, 1570, 3, 486, 216, 1052, 6, 1322, 3, 370, 6, 1684, 60, 26, 60, 104, 739, 6, 0, 47, 1, 5, 34, 186, 739, 6, 47, 213, 96, 15, 584, 1],
       [12, 2, 620, 82], [12, 2, 331, 817],
       [615, 352, 1124, 136, 122, 5, 33, 49, 55, 21, 156, 547, 150, 615, 11, 10, 145, 1191, 186, 12, 20, 15, 446, 7],
       [27, 1308, 0, 72, 2, 81, 41, 447, 136, 122, 541, 11, 82, 1688, 10, 136, 122, 541, 11, 728, 815, 10, 136, 122, 12, 20, 15, 446, 4, 41, 72, 7],
       [34, 17, 2, 0, 1303, 544, 17, 6, 82, 17],
       [1, 55, 0, 72, 2, 81, 41, 183, 1140, 183, 11, 10, 54, 183, 1045, 54, 183, 1044, 54, 1298, 1, 1060, 54, 12, 20, 446, 4, 1197, 7],
       [38, 440, 13, 597, 33, 24, 2, 763, 19, 148, 59, 7],
       [12, 2, 366, 22, 7]], dtype=object)

In [15]:
x_example, y_example = X_question[0], y_answer[0]
print("x:\n%s\n%s" % (" ".join([wordlist[x] for x in x_example]), x_example))
print("\ny:\n%s\n%s" % (" ".join([wordlist[x] for x in y_example]), y_example))

x:
in a program the code is this try { } finally { } whether program compiles & runs successfully ?
[5, 8, 184, 0, 72, 2, 41, 447, 136, 122, 878, 136, 122, 258, 184, 1111, 389, 362, 1228, 7]

y:
yes , try..finally can be possible without catch block
[140, 3, 1468, 13, 15, 161, 145, 541, 186]


In [16]:
import numpy as np

x = X_question
y = y_answer
max_length =  45
x_padded = []
y_padded = []


for row in x:
    if len(row) <= max_length:
        x_padded.append(row + [0] * (max_length - len(row)))
    else :
        x_padded.append(row[:45])

for row in y:
    if len(row) <= max_length:
        y_padded.append(row + [0] * (max_length - len(row))) 
    else: 
        y_padded.append(row[:45])


### Test Batch Pre-Processing

In [24]:
with open('Interviewtestdata_Question.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testquestions = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testquestions))

with open('Interviewtestdata_Answer.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testanswers = [nltk.word_tokenize(x[0].lower()) for x in reader]
    print(len(testanswers)) 

with open('Interviewtestdata_label.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    testlabels = np.array(list(itertools.chain(*[x for x in reader]))).astype(np.int32)
    print(len(testlabels))

4
4
4


In [25]:
for i, sent in enumerate(testquestions):
    testquestions[i] = [w if w in word_to_index else unknown_token for w in sent]
    
for i, sent in enumerate(testanswers):
    testanswers[i] = [w if w in word_to_index else unknown_token for w in sent]

In [26]:
X_testquestion = np.asarray([[word_to_index[w] for w in sent] for sent in testquestions])
y_testanswer = np.asarray([[word_to_index[w] for w in sent] for sent in testanswers])

In [27]:
max_length = 45
x_testpadded = []
y_testpadded = []
p = X_testquestion
q = y_testanswer

for row in p:
    if len(row) <= max_length:
        x_testpadded.append(row + [0] * (max_length - len(row))) 
    else :
        x_testpadded.append(row[:45])

for row in q:
    if len(row) <= max_length:
        y_testpadded.append(row + [0] * (max_length - len(row)))
    else :
        y_testpadded.append(row[:45])

In [28]:
testlabels

array([0, 0, 0, 0])

In [29]:
tf.reset_default_graph()

from tensorflow.contrib import rnn 
hm_epochs = 55
batch_size = 4
state_size = 200
vocab_size = 1720
num_steps = 45

# Placeholders
context = tf.placeholder(tf.int32, [batch_size, num_steps]) # [batch_size, num_steps]
utterance = tf.placeholder(tf.int32, [batch_size,num_steps])
targets = tf.placeholder(tf.int32, [batch_size])
target = tf.placeholder(tf.int32, [batch_size])

def next_batch(step):
    p,q = x_padded[batch_size*step:batch_size*(step+1)], y_padded[batch_size*step:batch_size*(step+1)]
    t = labels[batch_size*step:batch_size*(step+1)]
    return p,q,t

def test_batch(step):
    a,b = x_testpadded[batch_size*step:batch_size*(step+1)], y_testpadded[batch_size*step:batch_size*(step+1)]
    e = testlabels[batch_size*step:batch_size*(step+1)]
    return a,b,e
        
def dual_encoder_model(context,utterance,targets):
    
    #Embedding the question and answer
    context_embedded = tf.nn.embedding_lookup(embeddings, context, name="embed_context")
    utterance_embedded = tf.nn.embedding_lookup(embeddings, utterance, name="embed_utterance")
    print(" This is the embeddings for both Q&A:", context_embedded, utterance_embedded)
    
    with tf.variable_scope("rnn") as vs:
        cell = tf.contrib.rnn.BasicLSTMCell(state_size)
        cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = 0.5) 
        #cell = tf.contrib.rnn.MultiRNNCell(cells= [cell]*3, state_is_tuple= True)
    
    #feed inputs to dynamic rnn for training    
    rnn_inputs = tf.concat([context_embedded, utterance_embedded],0)
    rnn_inputs = tf.transpose(rnn_inputs, [1,0,2])
    rnn_inputs = tf.reshape(rnn_inputs, [-1, state_size])
    rnn_inputs = tf.split(rnn_inputs, num_steps, 0)
    
    rnn_outputs, rnn_states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float64)
    
    encoding_context, encoding_utterance = tf.split(rnn_states.h,2,0)
    
    print("\nThis is the encoding context:", encoding_context)
    print("\nThis is the encoding utterance:", encoding_utterance)
    
    with tf.variable_scope("prediction") as vs:
        M = tf.get_variable("M",
          shape=[state_size, state_size],
          initializer=tf.truncated_normal_initializer())

    # "Predict" a  response: c * M
    generated_response = tf.matmul(encoding_context, tf.cast(M, dtype = tf.float64))
    print("\nThis is the generated response:", generated_response)
    generated_response = tf.expand_dims(generated_response, 2)
    print("\nThis is the generated response:", generated_response)
    encoding_utterance = tf.expand_dims(encoding_utterance, 2)
    print("\nThis is the encoding utterance:", encoding_utterance)
    # Dot product between generated response and actual response
    # (c * M) * r
    logit = tf.matmul(generated_response, encoding_utterance, True)
    print("\nThis is the logits before squeeze:", logit)
    logits = tf.squeeze(logit, [2])
    print("\nThis is the logits after squeeze:", logits)

    # Apply sigmoid to convert logits to probabilities
    probs = tf.sigmoid(logits)   
    print("\nThis is the probability after sigmoid:", probs)
    return tf.reshape(logits, [-1]), tf.reshape(probs, [-1])

-

In [30]:
train_encoder(x)

 This is the embeddings for both Q&A: Tensor("embed_context:0", shape=(4, 45, 200), dtype=float64) Tensor("embed_utterance:0", shape=(4, 45, 200), dtype=float64)

This is the encoding context: Tensor("split_1:0", shape=(4, 200), dtype=float64)

This is the encoding utterance: Tensor("split_1:1", shape=(4, 200), dtype=float64)

This is the generated response: Tensor("MatMul:0", shape=(4, 200), dtype=float64)

This is the generated response: Tensor("ExpandDims:0", shape=(4, 200, 1), dtype=float64)

This is the encoding utterance: Tensor("ExpandDims_1:0", shape=(4, 200, 1), dtype=float64)

This is the logits before squeeze: Tensor("MatMul_1:0", shape=(4, 1, 1), dtype=float64)

This is the logits after squeeze: Tensor("Squeeze:0", shape=(4, 1), dtype=float64)

This is the probability after sigmoid: Tensor("Sigmoid:0", shape=(4, 1), dtype=float64)

This is the prediction size: Tensor("Reshape_1:0", shape=(4,), dtype=float64)

This is the target size: Tensor("Placeholder_2:0", shape=(4,), dt